# Chien luoc MA, ATR va mua 10 co phieu khi co tin hieu MUA

In [1]:
import yfinance as yf
import time 
import redis
import json
import pandas as pd
import numpy as np
import datetime

symbol = "VCB.VN"
start_date = "2023-10-01"
end_date = "2023-11-24"
ohlcv = yf.download(symbol, start=start_date, end=end_date, interval="15m")

# Chuyển dữ liệu thành DataFrame
data = pd.DataFrame(ohlcv, columns=['Open', 'High', 'Low', 'Close', 'Volume'])
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplcursors

# Tính MA và ATR
ma_fast_period = 50
ma_slow_period = 200
atr_period = 14
atr_threshold = 1000

data['MA_Fast'] = data['Close'].rolling(window=ma_fast_period).mean()
data['MA_Slow'] = data['Close'].rolling(window=ma_slow_period).mean()

# data['TR'] = np.maximum(data['High'] - data['Low'], np.maximum(abs(data['High'] - data['Close'].shift()), abs(data['Low'] - data['Close'].shift())))
# data['ATR'] = data['TR'].rolling(window=atr_period).mean()
data['TR'] = data['High'] - data['Low']
data['ATR'] = data['TR'].rolling(window=atr_period).mean()

# Xác định tín hiệu mua
data['Signal'] = np.where((data['MA_Fast'] > data['MA_Slow']) & (data['ATR'] <= atr_threshold), 1, 0)
data['Position'] = data['Signal'].shift()

data.to_csv('Buoi 9.1 - Backtest 1.csv')

# Giả sử khởi tạo với số vốn ban đầu là 100000000 và mỗi lần mua 10 cổ phiếu
initial_capital = 100000000
shares_per_signal = 10
capital = initial_capital
shares_held = 0

# Lặp qua mỗi hàng trong DataFrame
for index, row in data.iterrows():
    # Nếu có tín hiệu mua và có đủ vốn để mua
    if row['Position'] == 1 and capital >= row['Close'] * shares_per_signal:
        # Mua cổ phiếu và cập nhật vốn và số cổ phiếu được giữ
        capital -= row['Close'] * shares_per_signal
        shares_held += shares_per_signal
    # Cập nhật giá trị hiện tại của vốn dựa trên số cổ phiếu đang giữ và giá đóng cửa hiện tại
    current_value = capital + shares_held * row['Close']

# Tính lợi nhuận
profit = current_value - initial_capital

print(f'Tổng lợi nhuận: {profit}')
print(f'Tổng giá trị tài khoản: {current_value}')

[*********************100%***********************]  1 of 1 completed
Tổng lợi nhuận: -2347000.0
Tổng giá trị tài khoản: 97653000.0
